In [2]:
import os

from dotenv import load_dotenv
import pandas as pd
import numpy as np
import openai
import json
import time

### API Key - This key is private. You can get your own API key

In [3]:
load_dotenv()
API_KEY = os.environ['OPENAI_API_KEY']

## Adding tanacity implement exponential backoff to help with Rate Limit

In [4]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

### importing the dataset

In [5]:
main_data = pd.read_csv("../res/nasdaq100.csv")
num_data = pd.read_csv("../res/nasdaq100_price_change.csv")
main_data

,symbol,name,headQuarter,dateFirstAdded,cik,founded
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01
3,ADI,Analog Devices,"Wilmington, MA",NaN,6281,1965-01-01
4,ADP,ADP,"Roseland, NJ",NaN,8670,1949-01-01
...,...,...,...,...,...,...
96,WBD,Warner Bros. Discovery,"New York, NY",NaN,1437107,2022-04-08
97,WDAY,"Workday, Inc.","Pleasanton, CA",NaN,1327811,2005-03-01
98,XEL,Xcel Energy,"Minneapolis, MN",NaN,72903,1909-06-17
99,ZM,Zoom Video Communications,"San Jose, CA",NaN,1585521,2011-04-21


In [6]:
num_data

,symbol,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
0,AAPL,-1.7254,-8.30086,-6.20411,3.04200,15.64824,42.99992,8.479410,60.96299,245.42031,976.99441,139245.53954
1,ABNB,2.1617,-2.21919,9.88336,19.43286,19.64241,68.66902,23.640130,-1.04347,-1.04347,-1.04347,-1.04347
2,ADBE,0.5409,-1.77817,9.16191,52.04650,38.01522,57.22723,21.962060,17.83037,109.05718,1024.69214,251030.66399
3,ADI,0.9291,-4.03352,2.58486,3.65887,5.01602,17.02062,8.097350,63.42847,92.81874,286.77518,26012.63736
4,ADP,2.0589,2.35462,14.66581,16.40059,10.60546,5.53732,0.888943,81.76679,81.87224,248.40950,27613.11042
...,...,...,...,...,...,...,...,...,...,...,...,...
96,WBD,3.5791,13.98188,15.39075,12.25756,-5.60992,51.67715,3.209700,-31.02955,-43.36595,-65.85990,-12.30303
97,WDAY,1.4905,-1.04652,6.77548,28.69187,21.79922,37.90965,40.017990,31.59971,75.93220,221.38434,353.49515
98,XEL,-1.0432,-4.81045,-5.03734,-14.10091,-13.24042,-14.71386,-19.133960,-17.39010,26.61017,100.87395,724.27586
99,ZM,-0.5792,-3.99888,4.55307,8.29653,-18.89913,3.00030,-40.160360,-73.46268,10.74194,10.74194,10.74194


In [7]:
pd.merge(left=main_data,right=num_data['ytd'],how="left",on=main_data.symbol)

,key_0,symbol,name,headQuarter,dateFirstAdded,cik,founded,ytd
0,AAPL,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,42.99992
1,ABNB,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,68.66902
2,ADBE,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,57.22723
3,ADI,ADI,Analog Devices,"Wilmington, MA",NaN,6281,1965-01-01,17.02062
4,ADP,ADP,ADP,"Roseland, NJ",NaN,8670,1949-01-01,5.53732
...,...,...,...,...,...,...,...,...
96,WBD,WBD,Warner Bros. Discovery,"New York, NY",NaN,1437107,2022-04-08,51.67715
97,WDAY,WDAY,"Workday, Inc.","Pleasanton, CA",NaN,1327811,2005-03-01,37.90965
98,XEL,XEL,Xcel Energy,"Minneapolis, MN",NaN,72903,1909-06-17,-14.71386
99,ZM,ZM,Zoom Video Communications,"San Jose, CA",NaN,1585521,2011-04-21,3.00030


## See all companies present in the data

In [8]:
for comany_name in main_data['name']:
    print(comany_name)

Apple Inc.
Airbnb
Adobe Inc.
Analog Devices
ADP
Autodesk
American Electric Power
Align Technology
Applied Materials
Advanced Micro Devices Inc.
Amgen
Amazon
Ansys
ASML Holding
Activision Blizzard
Broadcom Inc.
AstraZeneca
Biogen
Booking Holdings
Baker Hughes
Cadence Design Systems
Constellation Energy
Charter Communications
Comcast
Costco
Copart
CrowdStrike
Cisco
CoStar Group
CSX Corporation
Cintas
Cognizant
Datadog
Dollar Tree
DexCom
Electronic Arts
eBay
Enphase Energy
Exelon
Diamondback Energy
Fastenal
Fortinet
GE HealthCare
GlobalFoundries
Gilead Sciences
Alphabet Inc. (Class C)
Alphabet Inc. (Class A)
Honeywell
Idexx Laboratories
Illumina, Inc.
Intel
Intuit
Intuitive Surgical
JD.com
Keurig Dr Pepper
Kraft Heinz
KLA Corporation
Lucid Motors
Lam Research
Lululemon
Marriott International
Microchip Technology
Mondelēz International
MercadoLibre
Meta Platforms
Monster Beverage
Moderna
Marvell Technology
Microsoft
Micron Technology
Netflix
Nvidia
NXP
Old Dominion Freight Line
Onsemi
O'Re

In [9]:
#creating a sector column for data
main_data['sector'] = 'None'
main_data

,symbol,name,headQuarter,dateFirstAdded,cik,founded,sector
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,None
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,None
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,None
3,ADI,Analog Devices,"Wilmington, MA",NaN,6281,1965-01-01,None
4,ADP,ADP,"Roseland, NJ",NaN,8670,1949-01-01,None
...,...,...,...,...,...,...,...
96,WBD,Warner Bros. Discovery,"New York, NY",NaN,1437107,2022-04-08,None
97,WDAY,"Workday, Inc.","Pleasanton, CA",NaN,1327811,2005-03-01,None
98,XEL,Xcel Energy,"Minneapolis, MN",NaN,72903,1909-06-17,None
99,ZM,Zoom Video Communications,"San Jose, CA",NaN,1585521,2011-04-21,None


### Building API gateway

#### Initialize the openAPI client

In [10]:
client = openai.OpenAI(
    api_key=API_KEY
)

#### Testing OpenAPI prompt

In [11]:
test_prompt = f"""Classify the company Apple Inc into one of the following sectors: Technology, Consumer, Cyclical, Industrial, Utilities, Healthcare, Marketing, Communication, Energy, Consumer, Defense, Real Estate, or Finance. Please only answer with one sector."""


### Using wait_random_exponential to avoid 'Too many Requests' Error

In [11]:
@retry(wait=wait_random_exponential(min=20,max=60),stop=stop_after_attempt(3))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

In [15]:
chat_completion = completion_with_backoff.create(
    messages=[
     {
         "role":"user",
         "content":test_prompt   
     }   
    ],
    model = "gpt-3.5-turbo-0125",
    temperature=0.0)

chat_completion

ChatCompletion(id='chatcmpl-8zrYDhPZ4ElLWCAF7IdFuPdnssEie', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Technology', role='assistant', function_call=None, tool_calls=None))], created=1709754549, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_2b778c6b35', usage=CompletionUsage(completion_tokens=1, prompt_tokens=56, total_tokens=57))

In [16]:
for chunks in chat_completion:
    print(chunks)

('id', 'chatcmpl-8zrYDhPZ4ElLWCAF7IdFuPdnssEie')
('choices', [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Technology', role='assistant', function_call=None, tool_calls=None))])
('created', 1709754549)
('model', 'gpt-3.5-turbo-0125')
('object', 'chat.completion')
('system_fingerprint', 'fp_2b778c6b35')
('usage', CompletionUsage(completion_tokens=1, prompt_tokens=56, total_tokens=57))


In [23]:
data_to_store = (chat_completion.choices[0].message.content).strip()
data_to_store

'Technology'

### Note
While using the an OpenAI API account with no payment method, it is quite frequent to encounter a 'RetryError' error. To counteract this, we will run the function of filling columns until there are no null values in the 'sector' field. When we encounter a 'RetryError' we will make our program wait for a few seconds to retry again. 

We will not fill the columns that have already been filled and will only fill columns that need filling

In [12]:
#This will find the rows with a 'sector' as None
n = "Cognizant"
if main_data.loc[main_data['name'] == n,'sector'].values[0] == 'None':
    print("Already classified")


Already classified


### Learnings and how the function was edited accordinly:
#### - Better loggings helps easy debugging the code
#### - Automating everything makes things easier - automate looping until total None fields is 0 and skip non-None fields
#### - if more than 50% of requests are resulting in error makes it useless to run the function. Therefore terminate it, save the results and try again later

In [11]:
total_data = main_data.loc[main_data['sector'] == "None"].shape[0]
errors = 0
while main_data.loc[main_data['sector'] == "None"].shape[0] != 0 and errors < total_data/2:
    total_data = main_data.loc[main_data['sector'] == "None"].shape[0]
    errors = 0
    print("Looping through the dataset to classify the companies into sectors. This may take a while. Please be patient.")
    print("No of None values in the dataset is:", main_data.loc[main_data['sector'] == "None"].shape[0])
    for row, company_name in enumerate(main_data['name']):
        print("At company: ", company_name, " in row: ", row)
        if main_data.loc[main_data['name'] == company_name,'sector'].values[0] == 'None':
            try:
            # The below line creates a prompt to input to API for every company_name in the dataset
                gpt_prompt = f"""Classify the company {company_name} into one of the following sectors: Technology, Consumer, Cyclical, Industrial, Utilities, Healthcare, Marketing, Communication, Energy, Consumer, Defense, Real Estate, or Finance. Please only answer with one sector"""

                chat_completion = client.chat.completions.create(
                    messages=[
                        {
                            "role":"user",
                            "content":gpt_prompt   
                        }   
                    ],
                    model = "gpt-3.5-turbo-0125",
                    temperature= 0.0
                )
                
                print("Successfully fetched data. Moving on to the next company.")
                #here we are storing the fetched sector from chatGPT
                fetched_sector = (chat_completion.choices[0].message.content).strip()
                
                #saving the values inti the main dataset
                main_data.loc[main_data['name'] == company_name , 'sector'] = fetched_sector
            except Exception as e:
                errors += 1
                print("Too many Requests, Skipping and trying again later")
                print(e)
                time.sleep(20)
                continue
        else:
            print(f"{company_name} already classified")
            
            
if main_data.loc[main_data['sector'] == "None"].shape[0] != 0:
    print("Error Rate has crossed 50%. Save now and try later")
    print(errors ,"errors in", total_data)
else:
    print(errors ,"errors in", total_data)
    print("Done!")
    
print("Cleaning and saving data")
main_data['sector'] = main_data['sector'].str.replace('\.','',regex=True)

print("Saving data to csv")
main_data.to_csv("../res/updated_nasdaq100.csv")
print("Saved")
main_data

Looping through the dataset to classify the companies into sectors. This may take a while. Please be patient.
No of None values in the dataset is: 101
At company:  Apple Inc.
Successfully fetched data. Moving on to the next company.
At company:  Airbnb
Successfully fetched data. Moving on to the next company.
At company:  Adobe Inc.
Successfully fetched data. Moving on to the next company.
At company:  Analog Devices
Successfully fetched data. Moving on to the next company.
At company:  ADP
Successfully fetched data. Moving on to the next company.
At company:  Autodesk
Successfully fetched data. Moving on to the next company.
At company:  American Electric Power
Successfully fetched data. Moving on to the next company.
At company:  Align Technology
Successfully fetched data. Moving on to the next company.
At company:  Applied Materials
Successfully fetched data. Moving on to the next company.
At company:  Advanced Micro Devices Inc.
Successfully fetched data. Moving on to the next comp

,symbol,name,headQuarter,dateFirstAdded,cik,founded,sector
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,Technology
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,Real Estate
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,Technology
3,ADI,Analog Devices,"Wilmington, MA",NaN,6281,1965-01-01,Technology
4,ADP,ADP,"Roseland, NJ",NaN,8670,1949-01-01,Technology
...,...,...,...,...,...,...,...
96,WBD,Warner Bros. Discovery,"New York, NY",NaN,1437107,2022-04-08,Communication
97,WDAY,"Workday, Inc.","Pleasanton, CA",NaN,1327811,2005-03-01,Technology
98,XEL,Xcel Energy,"Minneapolis, MN",NaN,72903,1909-06-17,Utilities
99,ZM,Zoom Video Communications,"San Jose, CA",NaN,1585521,2011-04-21,Technology


### The below was written for test. Used the below to update the fetch function
#### As you can see, sometimes we can have few unwanted datafields like '.' in the answer. So let's fix them

In [ ]:
main_data['sector'] = main_data['sector'].str.replace('\.','',regex=True)

In [11]:
for sectors in main_data['sector'].unique():
    print(sectors)

Technology
Real Estate
Utilities
Healthcare
Energy
Communication
Consumer
Industrial
Finance


### Now we export the data and save it somewhere so we can easily use it for later

In [13]:
main_data = pd.read_csv("../res/updated_nasdaq100.csv").drop(columns=['Unnamed: 0','dateFirstAdded'],axis=1)
main_data

,symbol,name,headQuarter,cik,founded,sector
0,AAPL,Apple Inc.,"Cupertino, CA",320193,1976-04-01,Technology
1,ABNB,Airbnb,"San Francisco, CA",1559720,2008-08-01,Real Estate
2,ADBE,Adobe Inc.,"San Jose, CA",796343,1982-12-01,Technology
3,ADI,Analog Devices,"Wilmington, MA",6281,1965-01-01,Technology
4,ADP,ADP,"Roseland, NJ",8670,1949-01-01,Technology
...,...,...,...,...,...,...
96,WBD,Warner Bros. Discovery,"New York, NY",1437107,2022-04-08,Communication
97,WDAY,"Workday, Inc.","Pleasanton, CA",1327811,2005-03-01,Technology
98,XEL,Xcel Energy,"Minneapolis, MN",72903,1909-06-17,Utilities
99,ZM,Zoom Video Communications,"San Jose, CA",1585521,2011-04-21,Technology


In [14]:
sectors_all = main_data["sector"].unique()
sectors_all

array(['Technology', 'Real Estate', 'Utilities', 'Healthcare', 'Energy',
       'Communication', 'Consumer', 'Industrial', 'Finance'], dtype=object)

### Converting the dataframe to dictionary to pass it to openAI API

In [15]:
print(temp_data)


NameError: name 'temp_data' is not defined

In [21]:
# The below line creates a prompt to input to API for every company_name in the dataset
gpt_prompt = f"""Use the dataset passed with the prompt in form of a list converted from a dataframe to a records using pandas,to_dict() method. 
Please return me the best sector to invest in. Just give me the answer in one word.
Dataset Dictionary : {temp_data}
"""

result = completion_with_backoff_better(
    messages=[
        {"role":"user",
         "content": gpt_prompt}
    ],
        model = "gpt-3.5-turbo-1106",
        temperature= 0.0
    )

print("The best sector to invest in is",result.choices[0].message.content)

The best sector to invest in is Communication


### The main issue that arrises here is that there is no data to predict which company is the best. Therefore let's join tables to make enough data

In [15]:
main_data = pd.read_csv("../res/updated_nasdaq100.csv").drop(columns=['Unnamed: 0','dateFirstAdded'],axis=1)
main_data = pd.merge(left=main_data,right=num_data,how="left",on=main_data.symbol)
main_data = main_data.sort_values(by='sector')
# Dropping 5Y. 10Y and Max column to avoid data limits
temp_data = main_data.drop(columns=['symbol_x','symbol_y'],axis=1)
temp_data

,key_0,name,headQuarter,cik,founded,sector,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
90,TMUS,T-Mobile US,"Bellevue, WA",1283699,2001-09-02,Communication,1.5230,-0.473416,-1.669430,-4.81332,-4.43388,-1.71893,-5.48485,18.73317,109.16884,469.13786,271.62685
23,CMCSA,Comcast,"Philadelphia, PA",1166691,1963-11-13,Communication,1.3604,-0.109890,9.703110,12.41652,14.57020,27.77622,17.77663,6.16678,28.38983,101.19522,46493.40209
87,SIRI,Sirius XM,"New York, NY",908937,2008-07-29,Communication,0.0874,-4.373760,8.089890,28.95442,-4.94071,-16.92573,-27.66917,-19.56522,-29.88338,27.08058,4.00000
70,NFLX,Netflix,"Los Gatos, CA",1065280,1997-09-29,Communication,2.1223,1.270590,0.607167,36.55967,21.43821,49.43550,88.77040,-10.90898,25.27641,1138.04120,36739.67863
22,CHTR,Charter Communications,"Stamford, CT",1091667,1993-01-01,Communication,0.1361,0.664811,12.768190,19.54681,4.17774,22.79115,-9.61534,-30.31219,41.20796,223.55935,1098.37143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,GOOGL,Alphabet Inc. (Class A),"Mountain View, CA",1652044,2015-10-02,Technology,2.6696,1.606800,10.085370,24.59032,22.19435,47.58752,12.13129,75.56411,109.46936,490.12389,5138.12918
100,ZS,Zscaler,"San Jose, CA",1713683,2007-01-01,Technology,-0.5610,-8.591190,1.508590,62.46367,5.27307,31.89945,-13.56527,15.48669,290.17450,340.42424,340.42424
6,AEP,American Electric Power,"Columbus, OH",4904,1906-01-01,Utilities,-0.4431,-3.588030,-4.045560,-12.19194,-12.35371,-14.74650,-19.18465,-6.60508,13.53172,77.87552,135.71585
38,EXC,Exelon,"Chicago, IL",1109357,2000-10-20,Utilities,0.1253,-2.914740,-2.416990,-6.72112,-2.08231,-7.39110,-11.13828,2.67146,-6.54664,29.60441,400.89778


In [16]:
passing = temp_data.loc[temp_data['sector'] == 'Technology']
passing = passing.to_dict(orient="records")
len(passing)

49

In [18]:
sectors_all = temp_data["sector"].unique()
sectors_all

array(['Communication', 'Consumer', 'Energy', 'Finance', 'Healthcare',
       'Industrial', 'Real Estate', 'Technology', 'Utilities'],
      dtype=object)

### Now let's run the model again with the function defined

In [19]:
@retry(wait=wait_random_exponential(min=30,max=60),stop=stop_after_attempt(3))
def completion_with_backoff_better(**kwargs):
    return client.chat.completions.create(**kwargs)

In [20]:
lis = []
for cur_sector in sectors_all:
    print("Now running for the sector:", cur_sector)
    pass_dict_specific = temp_data.loc[temp_data['sector'] == cur_sector]
    print("Size of the passed dictionary is:",pass_dict_specific.shape[0])
    try:
        print("Running the analysis")
        # The below line creates a prompt to input to API for every company_name in the dataset
        gpt_prompt = f"""Give me top 3 companies to invest in the passed dabaset based on the information. Make sure all the companies specified are in lowercase and the first letter is in uppercase.
        
        Give me the output in either above specified format or just give me 'NULL' with the reason for why the answer is NULL.
        If there are less than 3 companies in the sector, then just give me the company name as the answer

        Dataset : {pass_dict_specific}
        """

        result = completion_with_backoff_better(messages=[
            {
                "role":"system",
                "content":"""You will be given a dataset passed in the form of a list of dictionary. A single dictionary being a single record(row) of the dataset. The keys of the dictionary being the column of the dataset.
                The dictionary contains all data of companies in a given sector . You must use this dataset and numerical features to predict whatever is passed in the prompt.
                You must always return the data in the form of "1. company \n 2. company \n 3. company" arranged in the order of best to worst.
                """},
            {
                "role": "user",
                "content": gpt_prompt
            }
        ], model="gpt-3.5-turbo-1106",
            temperature=0.6)
        
        mess = result.choices[0].message.content
        lis.append(mess)
        
        print(f"The top 3 companies to invest in the {cur_sector} sector are: ")
        print(mess)
    except Exception as e:
        print("Analysis Failed! Too many Requests. Trying again later")
        print(e)
        time.sleep(20)
    print("--------------------------------")
        
    

Now running for the sector: Communication
Size of the passed dictionary is: 6
Running the analysis
The top 3 companies to invest in the Communication sector are: 
1. Netflix
2. Warner bros. discovery
3. Comcast
--------------------------------
Now running for the sector: Consumer
Size of the passed dictionary is: 12
Running the analysis
The top 3 companies to invest in the Consumer sector are: 
1. Marriott International 
2. Costco 
3. Starbucks
--------------------------------
Now running for the sector: Energy
Size of the passed dictionary is: 4
Running the analysis
The top 3 companies to invest in the Energy sector are: 
1. Diamondback Energy
2. Constellation Energy
3. Baker Hughes
--------------------------------
Now running for the sector: Finance
Size of the passed dictionary is: 2
Running the analysis
The top 3 companies to invest in the Finance sector are: 
1. Paypal
2. Paychex
--------------------------------
Now running for the sector: Healthcare
Size of the passed dictionary 

In [27]:
for l in lis:
    ll = l.split('\n')
    for na in ll:
        kk = na.split()
        kk.pop(0)
        na = " ".join(kk)
    
    print("----------------------------------------------------")

['1. Netflix', '2. Warner bros. discovery', '3. Comcast']
----------------------------------------------------
['1. Marriott International ', '2. Costco ', '3. Starbucks']
----------------------------------------------------
['1. Diamondback Energy', '2. Constellation Energy', '3. Baker Hughes']
----------------------------------------------------
['1. Paypal', '2. Paychex']
----------------------------------------------------
['1. Amgen ', '2. Gilead Sciences ', '3. Vertex Pharmaceuticals']
----------------------------------------------------
['1. Fastenal ', '2. Copart ', '3. Cintas']
----------------------------------------------------
['1. Airbnb', '2. Pdd Holdings', '3. CoStar Group']
----------------------------------------------------
['1. Nvidia ', '2. Booking Holdings ', '3. MercadoLibre']
----------------------------------------------------
['1. Aep', '2. Exelon', '3. Xel']
----------------------------------------------------


Now running for the sector: Communication
Size of the passed dictionary is: 6
Running the analysis
The top 3 companies to invest in the Communication sector are: 
Warner bros discovery 
Netflix 
Comcast
--------------------------------
Now running for the sector: Consumer
Size of the passed dictionary is: 12
Running the analysis
The top 3 companies to invest in the Consumer sector are: 
Starbucks 
Costco 
Marriott International
--------------------------------
Now running for the sector: Energy
Size of the passed dictionary is: 4
Running the analysis
